# PageRank

Adapted from [Cleve Moler's MATLAB book](https://www.mathworks.com/moler/exm/chapters/pagerank.pdf).

## Linear Algebra Primer

A common task in engineering is solving systems of linear equations. This example will demonstrate how to solve a simple set of linear equations using Google's PageRank algorithm as an example.

Consider the following system of equations:
    
\begin{align}
    ax_{1} + bx_{2} &= z_{1} \\
    cx_{1} + dx_{2} &= z_{2}
\end{align}

In linear algebra, this system is susually represented as follows:

\begin{align}
    \begin{bmatrix}
        a & b \\
        c & d
    \end{bmatrix}
    \begin{bmatrix}
        x_{1} \\
        x_{2}
    \end{bmatrix}
    =
    \begin{bmatrix}
        z_{1} \\
        z_{2}
    \end{bmatrix}
\end{align}


The system may be solved by multiplying the inverse of the coefficient matrix by the Z matrix

\begin{align}
    \begin{bmatrix}
        a & b \\
        c & d
    \end{bmatrix}
    ^{-1}
    \begin{bmatrix}
        z_{1} \\
        z_{2}
    \end{bmatrix}
    =
    \begin{bmatrix}
        x_{1} \\
        x_{2}
    \end{bmatrix}
\end{align}

We will now solve a simple word problem as a simple demonstration of this process.

> Stanley and Nancy are at the fair.

> Stanley buys **two burgers** and **five hotdogs** and pays a total of **$19**.

> Nancy buys **three burgers** and **seven hotdogs** and pays a total of **$27**.

> What is the price of of a single hamburger or hot dog?

We start by setting up the matrix using the information provided by the problem

\begin{align}
    \begin{bmatrix}
        2 & 5 \\
        3 & 7
    \end{bmatrix}
    ^{-1}
    \begin{bmatrix}
        19 \\
        27
    \end{bmatrix}
    =
    \begin{bmatrix}
        x_{1} \\
        x_{2}
    \end{bmatrix}
\end{align}

For a simple 2X2 mmatrix, we can solve for the inverse of a matrix using the following equation:

\begin{align}
    \begin{bmatrix}
        a & b \\
        c & d
    \end{bmatrix}^{-1}
    =
    \frac{1}{ad-cb}
    \begin{bmatrix}
        d & -b \\
        -c & a
    \end{bmatrix}
\end{align}

Plugging in the values:

\begin{align}
    \begin{bmatrix}
        2 & 5 \\
        3 & 7
    \end{bmatrix}^{-1}
    =
    \frac{1}{2\times7-3\times5}
    \begin{bmatrix}
        7 & -5 \\
        -3 & 2
    \end{bmatrix}
    =
    \begin{bmatrix}
        -7 & 5 \\
        2 & -2
    \end{bmatrix}
\end{align}

Multiplying the matrices we get the following result:

\begin{align}
    \begin{bmatrix}
        -7 & 5 \\
        2 & -2
    \end{bmatrix}
    \begin{bmatrix}
        19 \\
        27
    \end{bmatrix}
    &=
    \begin{bmatrix}
        x_{1} \\
        x_{2}
    \end{bmatrix}\\\\
    \begin{bmatrix}
        -7 & 5 \\
        2 & -2
    \end{bmatrix}
    \begin{bmatrix}
        19 \\
        27
    \end{bmatrix}
    &=
    \begin{bmatrix}
        x_{1} \\
        x_{2}
    \end{bmatrix}\\\\
    \begin{bmatrix}
        (-7\times19)+(5\times27) \\
        (3\times9)+(-2\times27)
    \end{bmatrix}
    &=
    \begin{bmatrix}
        2 \\
        3
    \end{bmatrix}\\\\
    \begin{bmatrix}
        x_{1} \\
        x_{2}
    \end{bmatrix}
    &=
    \begin{bmatrix}
        2 \\
        3
    \end{bmatrix}
\end{align}


**Therefore, burgers cost \$2 and hotdogs cost \$3**

---

Now that we have solved the problem by hand, let's solve it with Python.

In [5]:
import numpy as np

# assign our coefficients matrix
a = np.array([[2,5], [3,7]])

# assign out z matrix
z = np.array([19,27])

# use numpy's built in function for solving systems of equations
x = np.linalg.solve(a,z)

# print the results
print("Burgers cost: ${0:.2f}".format(x[0]))
print("Hotdogs cost: ${0:.2f}".format(x[1]))

Burgers cost: $2.00
Hotdogs cost: $3.00


## The Algebraic PageRank Algorithm

The first step in calculating PageRanks is setting up a connectivity matrix. In the real world this is data that would need to be collected by a web crawler, but here we will assume this has already been done.

Below is a diagram showing the way in which a set of pages link to each other. Each arrow represents a one-directional link from one page to another.

![connectivity diagram of webpages](http://i.imgur.com/ezYQQBd.png)

We can represent the connections in this diagram using a matrix. If we place the names of each website on each axis, we can put the number of links from one page to the other at the corresponding value. Here, the column index represents the page the link is on, and the row index indicates the destination of the link. We will refer to this connecticity matrix as **G**.

\begin{align}
    G =
    \begin{bmatrix}
        0 & 0 & 0 & 1 & 0 & 1 \\
        1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 1 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 \\
        1 & 0 & 1 & 0 & 0 & 0
    \end{bmatrix}
\end{align}


Now that we have a representation of the pages on our small web, we can look at what a PageRank actually is. The problem of computing PageRanks may be represented using the following equation:

\begin{align}
    \begin{bmatrix}
        \texttt{probability} \\
        \texttt{of} \\
        \texttt{clicking link}
    \end{bmatrix}
    \begin{bmatrix}
        \texttt{probability} \\
        \texttt{of} \\
        \texttt{being on a page}
    \end{bmatrix}
    &=
    \begin{bmatrix}
        1 \\
        \vdots \\
        1
    \end{bmatrix}
\end{align}

The first matrix, represents the probability of clicking a link from one page to another. This matrix will be calculated using information from the connectivity matrix created above.

The second matrix, really a column vector, is the probability of being on a given webpage. These are the PageRanks for each of the pages.

The final column vector of ones is representative of the fact that probabilities must sum to 1.

To construct the first matrix, we will look at the connectivity matrix. From the connectivity matrix, we determine that probability of going to a page *i* from a page *j* is given by the following equation:

\begin{align}
    p_{i} = \frac{1}{\texttt{# of links on page j}}
\end{align}

All this means is that if there are N links on a page, you have a $\frac{1}{N}$ chance of selecting a given page. Again, this assumes you are clicking links at random.

Using this probability, we will construct a weighting matrix, **W**, in the following manner.

\begin{align}
W=
    \begin{bmatrix}
        \frac{1}{N_{0}} & \dots & 0 \\
        \vdots & \ddots & \vdots \\
        0 & \dots & \frac{1}{N_{i}}
    \end{bmatrix}
\end{align}

This is a matrix that only has values along the diagonal. All other values are zero. The value on the diagonal is defined as the inverse of of the sum of the corresponding column in **G**, the connectivity matrix.

Next, we will define a damping factor that accounts for the fact users will not always click another link, and helps prevent sigularities from occuring when inverting the matrix.

\begin{align}
    d = \text{damping factor} \\
    0 < d < 1
\end{align}

As with most probability problems, we do not want *d* itself, but rather the probability $(1 - d)$. We will represent this as the identity matrix, **I**, minus the damping factor, *d*.

\begin{align}
    (I-d)
\end{align}



Now that we the necessary components, we will redefine the equation for the PageRank. Here, **R** will represent the column vector of page ranks, and **e** will represent the column vector of ones.

\begin{align}
    R =  \left[(I-d)G\times W \right]^{-1}e
\end{align}

Here we will implement a PageRank function in Python that has two inputs, a connectivity matrix and a damping factor. We will also set up some data to be fed to the function and print the result of the calculation.

In [6]:
import numpy as np

site_names = ['Alpha.com',
              'Bravo.com',
              'Charlie.com',
              'Delta.com',
              'Echo.com',
              'Foxtrot.com']

G = np.array([[0,0,0,1,0,1],
             [1,0,0,0,0,0],
             [0,1,0,0,0,0],
             [0,1,1,0,0,0],
             [0,0,1,0,0,0],
             [1,0,1,0,0,0]])

d = 0.85

def page_rank(G, d):
    
    # find the total number of pages
    N = len(G)
    
    # find the number of links on each page
    c = np.sum(G, axis=0)
    
    # create a weighting matrix based on number of links on each page
    with np.errstate(divide='ignore'): # watch out for divide by zero errors
        c = 1/c
        c[ ~ np.isfinite(c)] = 0 
    D = np.diag(c)

    # create a sparse identity matrix
    I = np.eye(N)
    
    # create a vector column of ones
    e = np.ones((N,1))
    
    # solve for the page ranks
    ranks = np.linalg.solve((I-d*np.dot(G,D)), e)
    ranks = ranks / np.sum(ranks)
    
    return ranks
	
R = page_rank(G, d)

print("Site Name\t\tPageRank")
print("----------\t\t----------")
for site in zip(site_names, R):
    print(str(site[0]) + '\t\t' + str(site[1]))

Site Name		PageRank
----------		----------
Alpha.com		[ 0.32101694]
Bravo.com		[ 0.17054304]
Charlie.com		[ 0.10659163]
Delta.com		[ 0.13679259]
Echo.com		[ 0.0643118]
Foxtrot.com		[ 0.200744]
